In [1]:
#%% 
import arviz as az
import matplotlib.pyplot as plt
plt.rcParams.update({'figure.constrained_layout.use':True})
import numpy as np
import pandas as pd
import pymc as pm
import os
import pytensor.tensor as pt
from scipy import stats

from scipy.stats import norm
from xarray_einstats.stats import XrContinuousRV

directory = os.fsencode("/Users/sydney/git/second-order-contacts/data/DataBimodalFit") #Please put absolute path here, when reproducing the fits
os.listdir(directory)

print('The pymc version is {}.' .format(pm.__version__))
#Comment: This notebook is based on/follows the structure of https://www.pymc.io/projects/examples/en/2021.11.0/mixture_models/gaussian_mixture_model.html

The pymc version is 5.10.4.


In [ ]:
for file in os.listdir(directory):
        
        if os.fsdecode(file) != ".DS_Store":
        
                filename = "/Users/sydney/git/second-order-contacts/data/DataBimodalFit/" + os.fsdecode(file) #Put absolute path here when reproducing the fits
                file = os.fsdecode(file)
                dataframe = pd.read_csv(
                        filename, index_col = 0
                )
                data = dataframe["value"].to_xarray()
                lendata = len(dataframe.index)

                #fixed_mean = dataframe['fixed_mean'].to_numpy()
                
                k = 1

                with pm.Model(coords={"cluster": np.arange(k), "obs_id": np.arange(lendata)}) as model_1:
                        sigma_1 = pm.Uniform("sigma_1", lower=1, upper=10)
                        #sigma_1 = pm.HalfCauchy("sigma_1", beta = 10)
                        #sigma_1 = pm.Gamma("sigma_1", alpha = 10, beta = 1)
                        points = pm.TruncatedNormal("obs", mu=-50, sigma=sigma_1, lower = -100, observed=data, dims="obs_id")
                
                pm.model_to_graphviz(model_1)
                
                with model_1:
                        trace1 = pm.sample(2000, model=model_1, return_inferencedata=True,cores=4, chains=4,idata_kwargs={"log_likelihood": True})
                        axes = az.plot_trace(trace1, var_names=["sigma_1"]);
                        
                        xi = np.linspace(-100, 75, 500)
                        post = trace1.posterior
                        summary = az.summary(post)
                        outputfilename = os.fsdecode(file) + "posteriorunimodal.csv"
                        summary.to_csv(outputfilename)
                        #First Component
                        fixed_mean_1 = -100
                        pdf_components_1 = XrContinuousRV(stats.truncnorm, loc=fixed_mean_1, scale=post["sigma_1"], a = -100, b =100).pdf(xi)
                        pdf_1 = pdf_components_1.sum() 
                        
                        fig, ax = plt.subplots(4, 1, figsize=(7, 12), sharex=True, layout="constrained")
                        # empirical histogram
                        ax[0].hist(data, bins = 35)
                        ax[0].set(title="Data", xlabel="Change of No. of Contacts (in percent)", ylabel="Frequency")
                        # pdf
                        pdf_components_1.mean(dim=["chain", "draw"]).plot.line(ax=ax[1])
                        ax[1].set(title="PDF", xlabel="Change of No. of Contacts (in percent)", ylabel="Probability\ndensity")
                        #Combination of histogram and pdf
                        ax[2].hist(data, bins = 35)
                        ax[2].set(title="Data", xlabel="Change of No. of Contacts (in percent)", ylabel="Frequency")
                        pdf_comb = pdf_components_1.mean(dim=["chain", "draw"])*5500
                        pdf_comb.plot.line(ax=ax[2])
                        ax[2].set(title="PDF", xlabel="Change of No. of Contacts (in percent)", ylabel="Frequency + Scaled Probability\ndensity")
                        # plot group membership probabilities
                        first_group = pdf_components_1.mean(dim=["chain", "draw"])/(pdf_components_1.mean(dim=["chain", "draw"]))
                        first_group.plot.line(ax=ax[3])
                        ax[3].set(title="Group membership", xlabel="Change of No. of Contacts (in percent)", ylabel="Probability");
                        
                        df = pd.DataFrame(first_group)
                        file_name = os.fsdecode(file) + "unimodalGroups.csv"
                        df.to_csv(file_name)
                        
                        fig_name = os.fsdecode(file) + "unimodalOutput.png"
                        fig.savefig(fig_name, bbox_inches="tight")
                        
                k = 2 #Fitting two distributions to the data

                with pm.Model(coords={"cluster": np.arange(k), "obs_id": np.arange(lendata)}) as model_2:
                        #cluster sizes
                        weights = pm.Dirichlet("weights", a=np.array([1,1]), dims="cluster", shape=(2))
                        #ensure all clusters have some points
                        category = pm.Categorical("category", p = weights, dims = "obs_id")
                        
                        sigma_1 = pm.Uniform("sigma_1", lower=1, upper=7)
                        sigma_2 = pm.Uniform("sigma_2", lower=1, upper=7)
                        
                        truncnorm_dist = pm.TruncatedNormal.dist(mu = -100, sigma = sigma_1, lower = -100)
                        norm_dist = pm.Normal.dist(mu = 0, sigma = sigma_2)
                        
                        components = [
                        truncnorm_dist,
                        norm_dist
                        ]
                        
                        points = pm.Mixture("obs", w=weights, comp_dists=components, observed=data, dims = "obs_id")
                
                pm.model_to_graphviz(model_2)
                
                with model_2:
                        step1 = pm.Metropolis(vars=[weights, sigma_1, sigma_2])
                        step2 = pm.CategoricalGibbsMetropolis(vars=[category])
                        trace2 = pm.sample(2000, model=model_2, step=[step1, step2], return_inferencedata=True,cores=4, chains=4,idata_kwargs={"log_likelihood": True})
                        axes = az.plot_trace(trace2, var_names=["weights", "sigma_1", "sigma_2"]);
                        
                        xi = np.linspace(-100, 75, 500)
                        post = trace2.posterior
                        summary = az.summary(post)
                        outputfilename = os.fsdecode(file) + "posteriorbimodal.csv"
                        summary.to_csv(outputfilename)
                        #First Component
                        fixed_mean_1 = -100
                        pdf_components_1 = XrContinuousRV(stats.truncnorm, loc=fixed_mean_1, scale=post["sigma_1"], a = -100, b =100).pdf(xi) * post["weights"].sel(cluster=0)
                        pdf_1 = pdf_components_1.sum() 
                        #Second Component
                        fixed_mean_2 = 0
                        pdf_components_2 = XrContinuousRV(norm, loc=fixed_mean_2, scale=post["sigma_2"]).pdf(xi) * post["weights"].sel(cluster=1)
                        pdf_2 = pdf_components_2.sum()

                        fig, ax = plt.subplots(4, 1, figsize=(7, 12), sharex=True, layout="constrained")
                        # empirical histogram
                        ax[0].hist(data, bins = 35)
                        ax[0].set(title="Data", xlabel="Change of No. of Contacts (in percent)", ylabel="Frequency")
                        # pdf
                        pdf_combination = pdf_components_1.mean(dim=["chain", "draw"]) + pdf_components_2.mean(dim=["chain", "draw"])
                        pdf_combination.plot.line(ax=ax[1])
                        ax[1].set(title="PDF", xlabel="Change of No. of Contacts (in percent)", ylabel="Probability\ndensity")
                        #Combination of histogram and pdf
                        ax[2].hist(data, bins = 35)
                        ax[2].set(title="Data", xlabel="Change of No. of Contacts (in percent)", ylabel="Frequency")
                        pdf_comb = pdf_combination*2000
                        pdf_comb.plot.line(ax=ax[2])
                        ax[2].set(title="PDF", xlabel="Change of No. of Contacts (in percent)", ylabel="Frequency + Scaled Probability\ndensity")
                        # plot group membership probabilities
                        first_group = pdf_components_1.mean(dim=["chain", "draw"])/(pdf_components_1.mean(dim=["chain", "draw"])+pdf_components_2.mean(dim=["chain", "draw"]))
                        first_group.plot.line(ax=ax[3])
                        second_group = pdf_components_2.mean(dim=["chain", "draw"])/(pdf_components_1.mean(dim=["chain", "draw"])+pdf_components_2.mean(dim=["chain", "draw"]))
                        second_group.plot.line(ax=ax[3])
                        ax[3].set(title="Group membership", xlabel="Change of No. of Contacts (in percent)", ylabel="Probability");
                        
                        df = pd.DataFrame({
                                "first_group": first_group,
                                "second_group": second_group}
                                )
                        file_name = os.fsdecode(file) + "bimodalGroups.csv"
                        df.to_csv(file_name)
                        
                        fig_name = os.fsdecode(file) + "bimodalOutput.png"
                        fig.savefig(fig_name, bbox_inches="tight")
                 
                k = 3 # Fitting 3 distributions to the data

                with pm.Model(coords={"cluster": np.arange(k), "obs_id": np.arange(lendata)}) as model_3:
                        #cluster sizes
                        weights = pm.Dirichlet("weights", a=np.array([1,1,1]), dims=("cluster"), shape=(3))
   
                        sigma_1 = pm.Uniform("sigma_1", lower=1, upper=7)
                        sigma_2 = pm.Uniform("sigma_2", lower=1, upper=15)
                        sigma_3 = pm.Uniform("sigma_3", lower=1, upper=7)
                        
                        truncnorm_dist = pm.TruncatedNormal.dist(mu = -100, sigma = sigma_1, lower = -100)
                        norm_dist_1 = pm.Normal.dist(mu = -50, sigma = sigma_2)
                        norm_dist_2 = pm.Normal.dist(mu = 0, sigma = sigma_3)
                        
                        components = [
                        truncnorm_dist,
                        norm_dist_1,
                        norm_dist_2
                        ]
                        
                        points = pm.Mixture("obs", w=weights, comp_dists=components, observed=data, dims = "obs_id")
                pm.model_to_graphviz(model_3)

                with model_3:
                        step1 = pm.Metropolis(vars=[weights, sigma_1, sigma_2, sigma_3])
                        #step2 = pm.CategoricalGibbsMetropolis(vars=[category])
                        trace3 = pm.sample(2000, model=model_3, step=[step1], return_inferencedata=True,cores=4, chains=4,idata_kwargs={"log_likelihood": True})
                        axes = az.plot_trace(trace3, var_names=["weights", "sigma_1", "sigma_2", "sigma_3"]);
                        
                        xi = np.linspace(-100, 75, 500)
                        post = trace3.posterior
                        summary = az.summary(post)
                        outputfilename = os.fsdecode(file) + "posteriortrimodal.csv"
                        summary.to_csv(outputfilename)
                        #First Component
                        fixed_mean_1 = -100
                        pdf_components_1 = XrContinuousRV(stats.truncnorm, loc=fixed_mean_1, scale=post["sigma_1"], a = -100, b =100).pdf(xi) * post["weights"].sel(cluster=0)
                        #Second Component
                        fixed_mean_2 = -50
                        pdf_components_2 = XrContinuousRV(norm, loc=fixed_mean_2, scale=post["sigma_2"]).pdf(xi) * post["weights"].sel(cluster=1)
                        #Third Component
                        fixed_mean_3 = 0
                        pdf_components_3 = XrContinuousRV(norm, loc=fixed_mean_3, scale=post["sigma_3"]).pdf(xi) * post["weights"].sel(cluster=2)

                        fig, ax = plt.subplots(4, 1, figsize=(7, 12), sharex=True, layout="constrained")
                        # empirical histogram
                        ax[0].hist(data, bins = 35)
                        ax[0].set(title="Data", xlabel="Change of No. of Contacts (in percent)", ylabel="Frequency")
                        # pdf
                        pdf_combination = pdf_components_1.mean(dim=["chain", "draw"]) + pdf_components_2.mean(dim=["chain", "draw"]) + pdf_components_3.mean(dim=["chain", "draw"])
                        pdf_combination.plot.line(ax=ax[1])
                        ax[1].set(title="PDF", xlabel="Change of No. of Contacts (in percent)", ylabel="Probability\ndensity")
                        #Combination of histogram and pdf
                        ax[2].hist(data, bins = 35)
                        ax[2].set(title="Data", xlabel="Change of No. of Contacts (in percent)", ylabel="Frequency")
                        pdf_comb = pdf_combination*2000
                        pdf_comb.plot.line(ax=ax[2])
                        ax[2].set(title="PDF", xlabel="Change of No. of Contacts (in percent)", ylabel="Frequency + Scaled Probability\ndensity")
                        # plot group membership probabilities
                        first_group = pdf_components_1.mean(dim=["chain", "draw"])/(pdf_components_1.mean(dim=["chain", "draw"])+pdf_components_2.mean(dim=["chain", "draw"])+pdf_components_3.mean(dim=["chain", "draw"]))
                        first_group.plot.line(ax=ax[3])
                        second_group = pdf_components_2.mean(dim=["chain", "draw"])/(pdf_components_1.mean(dim=["chain", "draw"])+pdf_components_2.mean(dim=["chain", "draw"])+pdf_components_3.mean(dim=["chain", "draw"]))
                        second_group.plot.line(ax=ax[3])
                        third_group = pdf_components_3.mean(dim=["chain", "draw"])/(pdf_components_1.mean(dim=["chain", "draw"])+pdf_components_2.mean(dim=["chain", "draw"])+pdf_components_3.mean(dim=["chain", "draw"]))
                        third_group.plot.line(ax=ax[3])
                        ax[3].set(title="Group membership", xlabel="Change of No. of Contacts (in percent)", ylabel="Probability");
                        
                        fig_name = os.fsdecode(file) + "trimodalOutput.png"
                        fig.savefig(fig_name, bbox_inches="tight")
                        
                        df = pd.DataFrame({
                                "first_group": first_group,
                                "second_group": second_group,
                                "third_group": third_group}
                                )
                        file_name = os.fsdecode(file) + "trimodalGroups.csv"
                        df.to_csv(file_name)
                        

                k = 4 #Fitting 4 distributions to the data

                with pm.Model(coords={"cluster": np.arange(k), "obs_id": np.arange(lendata)}) as model_4:
                        #cluster sizes
                        weights = pm.Dirichlet("weights", a=np.array([1,1,1,1]), dims=("cluster"), shape=(4))
                        
                        sigma_1 = pm.Uniform("sigma_1", lower=1, upper=7)
                        sigma_2 = pm.Uniform("sigma_2", lower=1, upper=15)
                        sigma_3 = pm.Uniform("sigma_3", lower=1, upper=7)
                        sigma_4 = pm.Uniform("sigma_4", lower=1, upper=15)
                        
                        truncnorm_dist = pm.TruncatedNormal.dist(mu = -100, sigma = sigma_1, lower = -100)
                        norm_dist_1 = pm.Normal.dist(mu = -50, sigma = sigma_2)
                        norm_dist_2 = pm.Normal.dist(mu = 0, sigma = sigma_3)
                        norm_dist_3 = pm.Normal.dist(mu = 50, sigma = sigma_4)
                        
                        components = [
                        truncnorm_dist,
                        norm_dist_1,
                        norm_dist_2,
                        norm_dist_3
                        ]
                        
                        points = pm.Mixture("obs", w=weights, comp_dists=components, observed=data, dims = "obs_id")

                with model_4:
                        step1 = pm.Metropolis(vars=[weights, sigma_1, sigma_2, sigma_3, sigma_4])
                        trace4 = pm.sample(2000, model=model_4, step=[step1], return_inferencedata=True,cores=4, chains=4,idata_kwargs={"log_likelihood": True})
                        axes = az.plot_trace(trace4, var_names=["weights", "sigma_1", "sigma_2", "sigma_3", "sigma_4"]);
                        
                        xi = np.linspace(-100, 75, 500)
                        post = trace4.posterior
                        summary = az.summary(post)
                        outputfilename = os.fsdecode(file) + "posteriorquatromodal.csv"
                        summary.to_csv(outputfilename)
                        #First Component
                        fixed_mean_1 = -100
                        pdf_components_1 = XrContinuousRV(stats.truncnorm, loc=fixed_mean_1, scale=post["sigma_1"], a = -100, b =100).pdf(xi) * post["weights"].sel(cluster=0)
                        #Second Component
                        fixed_mean_2 = -50
                        pdf_components_2 = XrContinuousRV(norm, loc=fixed_mean_2, scale=post["sigma_2"]).pdf(xi) * post["weights"].sel(cluster=1)
                        #Third Component
                        fixed_mean_3 = 0
                        pdf_components_3 = XrContinuousRV(norm, loc=fixed_mean_3, scale=post["sigma_3"]).pdf(xi) * post["weights"].sel(cluster=2)
                        #Fourth Component
                        fixed_mean_4 = 50
                        pdf_components_4 = XrContinuousRV(norm, loc=fixed_mean_4, scale=post["sigma_4"]).pdf(xi) * post["weights"].sel(cluster=3)


                        fig, ax = plt.subplots(4, 1, figsize=(7, 12), sharex=True, layout="constrained")
                        # empirical histogram
                        ax[0].hist(data, bins = 35
                                   )
                        ax[0].set(title="Data", xlabel="Change of No. of Contacts (in percent)", ylabel="Frequency")
                        # pdf
                        pdf_combination = pdf_components_1.mean(dim=["chain", "draw"]) + pdf_components_2.mean(dim=["chain", "draw"]) + pdf_components_3.mean(dim=["chain", "draw"]) + pdf_components_4.mean(dim=["chain", "draw"])
                        pdf_combination.plot.line(ax=ax[1])
                        ax[1].set(title="PDF", xlabel="Change of No. of Contacts (in percent)", ylabel="Probability\ndensity")
                        #Combination of histogram and pdf
                        ax[2].hist(data, bins = 35)
                        ax[2].set(title="Data", xlabel="Change of No. of Contacts (in percent)", ylabel="Frequency")
                        pdf_comb = pdf_combination*2000
                        pdf_comb.plot.line(ax=ax[2])
                        ax[2].set(title="PDF", xlabel="Change of No. of Contacts (in percent)", ylabel="Frequency + Scaled Probability\ndensity")
                        # plot group membership probabilities
                        first_group = pdf_components_1.mean(dim=["chain", "draw"])/(pdf_components_1.mean(dim=["chain", "draw"])+pdf_components_2.mean(dim=["chain", "draw"])+pdf_components_3.mean(dim=["chain", "draw"])+pdf_components_4.mean(dim=["chain", "draw"]))
                        first_group.plot.line(ax=ax[3])
                        second_group = pdf_components_2.mean(dim=["chain", "draw"])/(pdf_components_1.mean(dim=["chain", "draw"])+pdf_components_2.mean(dim=["chain", "draw"])+pdf_components_3.mean(dim=["chain", "draw"])+pdf_components_4.mean(dim=["chain", "draw"]))
                        second_group.plot.line(ax=ax[3])
                        third_group = pdf_components_3.mean(dim=["chain", "draw"])/(pdf_components_1.mean(dim=["chain", "draw"])+pdf_components_2.mean(dim=["chain", "draw"])+pdf_components_3.mean(dim=["chain", "draw"])+pdf_components_4.mean(dim=["chain", "draw"]))
                        third_group.plot.line(ax=ax[3])
                        fourth_group = pdf_components_4.mean(dim=["chain", "draw"])/(pdf_components_1.mean(dim=["chain", "draw"])+pdf_components_2.mean(dim=["chain", "draw"])+pdf_components_3.mean(dim=["chain", "draw"])+pdf_components_4.mean(dim=["chain", "draw"]))
                        fourth_group.plot.line(ax=ax[3])
                        ax[3].set(title="Group membership", xlabel="Change of No. of Contacts (in percent)", ylabel="Probability");
                        
                        df = pd.DataFrame({
                                "first_group": first_group,
                                "second_group": second_group,
                                "third_group": third_group,
                                "fourth_group":fourth_group}
                                )
                        file_name = os.fsdecode(file) + "quatromodalGroups.csv"
                        df.to_csv(file_name)
                        
                        fig_name = os.fsdecode(file) + "quatromodalOutput.png"
                        fig.savefig(fig_name, bbox_inches="tight")
                
                # Comparison of models via leave-one-out cross validation 
                df_comp_loo = az.compare({"one_dist":trace1,  "two_dist":trace2, "three_dist":trace3,  "four_dist":trace4})
                outputfilename =  os.fsdecode(file) + "loo_comparison.csv"
                df_comp_loo.to_csv(outputfilename)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x10da0b850>>
Traceback (most recent call last):
  File "/Users/sydney/git/second-order-contacts/myenv/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


,first_group,second_group,third_group
0,0.994429,0.005571,0.0
1,0.993993,0.006007,0.0
2,0.992688,0.007312,0.0
3,0.989960,0.010040,0.0
4,0.984468,0.015532,0.0
...,...,...,...
495,0.000000,1.000000,0.0
496,0.000000,1.000000,0.0
497,0.000000,1.000000,0.0
498,0.000000,1.000000,0.0
